# Acknolwedgements

Massive thanks to Brev for publishing this amazing article and explanation of the notebook!

https://github.com/brevdev/notebooks/blob/main/mistral-finetune-own-data.ipynb

# Check GPU

In [1]:
!nvidia-smi

Tue Nov  7 13:42:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install Libraries

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.0 MB/s eta 0:00

# Load and Format Dataset

In [3]:
from datasets import load_dataset

train_dataset = load_dataset("weaviate/WithRetrieval-Random-Train-80")
eval_dataset = load_dataset("weaviate/WithRetrieval-Random-Test-80")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
def formatting_func(example):
    text = f"""
    ## Instruction
    Your task is to write GraphQL for the Natural Language Query provided.
    Use the provided API reference and Schema to generate the GraphQL.
    Only use the API reference to understand the syntax of the Request.

    ## Natural Language Query
    {example["nlcommand"]}

    ## Schema
    {example["schema"]}

    ## API Reference
    {example["apiRef"]}

    ## Answer
    {example["output"]}
    """
    return text

# Load Model

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#base_model_id = "mistralai/Mistral-7B-v0.1"
base_model_id = "Deci/DeciCoder-1b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

The repository for Deci/DeciCoder-1b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Deci/DeciCoder-1b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/Deci/DeciCoder-1b:
- configuration_decicoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for Deci/DeciCoder-1b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Deci/DeciCoder-1b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/Deci/DeciCoder-1b:
- modeling_decicoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [6]:
model.config

DeciCoderConfig {
  "_name_or_path": "Deci/DeciCoder-1b",
  "architectures": [
    "DeciCoderForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciCoder-1b--configuration_decicoder.DeciCoderConfig",
    "AutoModelForCausalLM": "Deci/DeciCoder-1b--modeling_decicoder.DeciCoderForCausalLM"
  },
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5888,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "naive_attention_decode_batched": true,
  "naive_attention_decode_single": false,
  "naive_attention_prefill": false,
  "num_attention_heads": 32,
  "num_hidden_layers": 20,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_i

In [7]:
model.config

DeciCoderConfig {
  "_name_or_path": "Deci/DeciCoder-1b",
  "architectures": [
    "DeciCoderForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciCoder-1b--configuration_decicoder.DeciCoderConfig",
    "AutoModelForCausalLM": "Deci/DeciCoder-1b--modeling_decicoder.DeciCoderForCausalLM"
  },
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5888,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "naive_attention_decode_batched": true,
  "naive_attention_decode_single": false,
  "naive_attention_prefill": false,
  "num_attention_heads": 32,
  "num_hidden_layers": 20,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_i

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [8]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3190 [00:00<?, ? examples/s]

Map:   0%|          | 0/798 [00:00<?, ? examples/s]

In [9]:
max_length = 512

def generate_and_tokenize_prompt2(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt2)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt2)

Map:   0%|          | 0/3190 [00:00<?, ? examples/s]

Map:   0%|          | 0/798 [00:00<?, ? examples/s]

In [11]:
print(tokenized_train_dataset["train"][1]['input_ids'])

[284, 5286, 21081, 284, 10604, 2899, 438, 372, 2866, 19083, 436, 322, 42022, 6633, 6645, 3943, 32, 284, 4417, 322, 3943, 2330, 4121, 461, 9042, 372, 4450, 322, 19083, 32, 284, 9287, 793, 322, 2330, 4121, 372, 7648, 322, 8345, 432, 322, 4448, 32, 446, 5286, 42022, 6633, 6645, 284, 3970, 878, 203, 4311, 597, 322, 3226, 282, 558, 30, 3048, 30, 461, 7533, 432, 322, 4306, 18801, 43415, 4684, 32, 203, 914, 446, 5286, 9042, 284, 301, 203, 20, 4820, 563, 428, 203, 109, 203, 20, 823, 563, 313, 1512, 18801, 43415, 392, 203, 20, 2310, 563, 313, 1045, 963, 18801, 3226, 282, 1623, 4414, 203, 20, 2402, 1284, 620, 563, 313, 12699, 1586, 392, 203, 20, 2402, 2265, 563, 313, 878, 36, 3137, 31, 47718, 392, 203, 20, 4642, 563, 428, 203, 109, 203, 20, 426, 563, 313, 33229, 558, 392, 203, 20, 23327, 563, 3037, 878, 3439, 203, 20, 2310, 563, 313, 558, 432, 322, 3226, 282, 3107, 203, 835, 203, 109, 203, 20, 426, 563, 313, 2310, 392, 203, 20, 23327, 563, 3037, 878, 3439, 203, 20, 2310, 563, 313, 3234, 432, 322

In [12]:
tokenized_train_dataset["train"][1]["input"]

'\n                Your task is to write an API request for a custom database schema based on the API reference provided.\n\n                For guidance on how to correctly format this API request, consult the API reference here:\n                Note: Please only use the API reference to understand the syntax of the request. Make sure your request is compliant with it.\n                Here are some quick notes about the API syntax:\n                - All queries should start with either `Get` or `Aggregate`. A common mistake is to begin the API request with `query`, please do not make this mistake.\n                - All queries should begin with an open curly bracket, `{`\n\n                API REFERENCE:\n                Simple Get to return a multiple properties\n\n`Get`\xa0function requirements\nTo retrieve objects from Weaviate, you must use the\xa0`Get`\xa0function and specify at least:\n\n- The target\xa0`class`\xa0to search, and\n- One or more\xa0`properties`\xa0to retrieve.

# Test Base Model

# Setup LoRA

In [13]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [15]:
print(model)

DeciCoderForCausalLM(
  (model): DeciCoderModel(
    (embed_tokens): Embedding(49152, 2048)
    (layers): ModuleList(
      (0-19): 20 x DeciCoderDecoderLayer(
        (self_attn): DeciCoderAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5888, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5888, bias=False)
          (down_proj): Linear4bit(in_features=5888, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (nor

In [16]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 25067520 || all params: 682608640 || trainable%: 3.6723121465324553


In [17]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): DeciCoderForCausalLM(
      (model): DeciCoderModel(
        (embed_tokens): Embedding(49152, 2048)
        (layers): ModuleList(
          (0-19): 20 x DeciCoderDecoderLayer(
            (self_attn): DeciCoderAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
              )
              (k_proj): Linear4bit(
                (lora_dropout): Mod

In [18]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [19]:
model = accelerator.prepare_model(model)

# Training

In [20]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [21]:
import transformers
from datetime import datetime

project = "weaviate-gorilla"
base_model_name = "decicoder"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset["train"],
    eval_dataset=tokenized_val_dataset["train"],
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=1500,
        learning_rate=2.5e-5, # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=50,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,1.218000,0.850673
100,0.735700,0.675857
150,0.639800,0.570352
200,0.537300,0.481115
250,0.421200,0.406780
300,0.383100,0.348370
350,0.344700,0.295756
400,0.275500,0.261340
450,0.248200,0.235128
500,0.228400,0.211407


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss,Validation Loss
50,1.218000,0.850673
100,0.735700,0.675857
150,0.639800,0.570352
200,0.537300,0.481115
250,0.421200,0.406780
300,0.383100,0.348370
350,0.344700,0.295756
400,0.275500,0.261340
450,0.248200,0.235128
500,0.228400,0.211407


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=1500, training_loss=0.25114797178904213, metrics={'train_runtime': 1061.386, 'train_samples_per_second': 2.826, 'train_steps_per_second': 1.413, 'total_flos': 9566907531264000.0, 'train_loss': 0.25114797178904213, 'epoch': 0.94})

In [23]:
model.save_pretrained("weaviate-gorilla-mistral-7b-1500-steps")

In [26]:
print("connection test")

connection test


# Evaluate the Model!

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)